# Capstone Final Project-Battle of the Neighborhoods-wk1
By Lian Wang

July 2020

## Table of contents

* [Introduction](#Introduction)
* [Data](#Data)
* [Methodology](#Methodology)
* [Results](#Results)
* [Discussions](#Discussions)
* [Conclusions](#Conclusions)


## Introduction <a name=Introduction></a>

In this age of convenient and efficient tranportation means, travelling around the world becomes easier and more common. In fact, people relocate around the world more often too with the globalization trend. 

When people move from one city to another, it is common to simply pick an area/neighborhood near work/school first, then move to a different area/neighbor if desired after settling down a bit. Moving, sometimes several times, isn't unusual in this scenario. It is obviously not an optimal process since we are only gathering limited/scattered information either via word of mouth or checking out a few nearby areas in person. Also, it would seem better to reasearch the new city and compare it to the city we currently live in to identify areas that might be a good fit for us in the new city, in advance. It would potentially minimize the need or frequency of relocations, which is a big hassle for those relocating with a family.

However, most of the times, efforts to research a new city only offer the city-level information, for example, population, histories, economic condition, climates, etc. It gives the overall picture of the city, which is more useful for tourists but not for selecting an area for living. For the latter, it will be more helpful to glean and compare neighborhood-level details that affect the day-to-day living.

In this project, we intend to take advantage of the rich location data provided by platforms like **Foursquare** and the power of machine learning to undertake comparisons of neighborhoods in two cities, using New York City and Toronto as the example. As described above, applications like this could be helpful for people who need to relocate to a new city or people choosing between cities for their next chapter in life.




### Data <a name=Data></a>

In order to use the **Foursquare** platform to gather neighborhood venue information, we need data that contains the neighborhoods exist in each city as well as the latitude and logitude coordinates of each neighborhood. 

For New York city, this data had been compiled and exists in one file at https://cocl.us/new_york_dataset for this IBM course. The original source of this data is from https://geo.nyu.edu/catalog/nyu_2451_34572.

For Toronto, the list of neighborhood and corresponding postal code will be scraped from this Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. Even though we could use geocoder Python package to retrieve the geographical location data based on postal codes, this package is unreliable (could get stuck in the process for unreasonably long time if using a while loop to ensure getting a result for each postal code). So, we will use the csv file containing the geographical location data for each of the postal code in Toronto that is provided for this IBM course at http://cocl.us/Geospatial_data.

Once we have the neighborhood data with the appropriate geographical data for both cities, we will then use the **Foursquare API** to retrieve the venues infromation within a certain range of the rarius (say, 500 or 1000 meters) for each neighborhood. Service and activitiy venues, nearby within a neighborhood, are characteristics of a neighborhood and reflect the convenience and life style of people living in the area. Hence, quantifying these venues into categories and the associagted venue counts are meaningful features to use for classifying neighborhoods into clusters/groups. Because our purpose is to compare the two cities, we will compile a combined data set for clustering analysis based on neighborhood venue features, and then examine the distribution of the clusters/groups between the two cities.



### Methodology <a name=Methodology></a>

We will first install packages and load the necessary libraries.

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line once installed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
#from pandas import json_normalize # for a newer Python version?

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line once installed
import folium # map rendering library

print('Libraries imported.')

### Load and extract neighborhood data for New York City

In [ ]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = data['properties']['borough'] # 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head()

In [ ]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Load and create neighborhood data for Toronto

### Using Foursquare API to retrieve the venues data for neighborhoods

### Results and Discussions <a name=Results></a>

### Discussions <a name=Discussions><a/>

### Conclusions<a name=Conclusions><a/>